In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('tidy_2.csv')

## 새로운 칼럼 만들기

### 1) 대상인원 칼럼 만들기

In [ ]:
df['대상인원'] = df['대상인원'].copy().str.extract(r'[\s|외|총|등](\d+)\s?[명|인]?')
# 대상인원이 없는 경우 운전기사 회식, 격려금 지급 등 맛집과 상관없는 경우가 많으니 삭제
df = df.dropna(axis=0, subset=['대상인원'], how='all')
df['대상인원'] = df['대상인원'].copy().astype(int)


### 2) 년도별로 그룹화하기

In [ ]:
df_2016 = df[df['year'] == 2016]
df_2017 = df[df['year'] == 2017]
df_2018 = df[df['year'] == 2018]

dfs = [df, df_2016, df_2017, df_2018]
df_2016.shape, df_2017.shape, df_2018.shape

### 3) 집행지명으로 그룹화하기

In [ ]:
def group_place(df):
    df_group = df.groupby('집행지명')\
            ['집행지명', 'cnt', '집행금액', '대상인원']\
            .sum().reset_index()
    return df_group

df_2016_group = group_place(df_2016)
df_2017_group = group_place(df_2017)
df_2018_group = group_place(df_2018)
df_group = group_place(df)

df_groups = [df_group, df_2016_group, df_2017_group, df_2018_group]
df_group.shape, df_2016_group.shape, df_2017_group.shape, df_2018_group.shape

In [ ]:
# 회당, 인당 금액을 구해보자. 
def price_per_num_people(df_group):
    df_group['회당집행금액'] = df_group['집행금액'] / df_group['cnt']
    df_group['인당집행금액'] = df_group['집행금액'] / df_group['대상인원']
    df_group['회당명수'] = df_group['대상인원'] / df_group['cnt']

for df_group in df_groups:
    df_group = price_per_num_people(df_group)

df_groups[0].head()

# 서울시 진짜 맛집 프로젝트

## 카테고리별 맛집

### 1. 가장 자주 간 식당 (전체 공무원)

In [ ]:
def by_cnt(df_group):
    df_group = df_group.sort_values(by='cnt', ascending=False)
    df_group['회당명수'] = round(df_group['회당명수']).astype(int)
    df_group['회당집행금액'] = round(df_group['회당집행금액']).astype(int)
    df_group['인당집행금액'] = round(df_group['인당집행금액']).astype(int)
    df_group['집행금액'] = round(df_group['집행금액']).astype(int)
    return df_group

for i, df_group in enumerate(df_groups):
    df_groups[i] = by_cnt(df_group)

In [ ]:
# 전체 공무원
df_groups[0].head(3)

In [ ]:
# 2016년
df_groups[1].head(3)

In [ ]:
# 2017년
df_groups[2].head(3)

In [ ]:
# 2018년
df_groups[3].head(3)

### 2. 가장 돈을 많이 쓴 식당 

In [ ]:
# 전체
df_groups[0].sort_values(by='집행금액', ascending=False).head(3)

In [ ]:
# 2016년
df_groups[1].sort_values(by='집행금액', ascending=False).head(3)

In [ ]:
# 2017년
df_groups[2].sort_values(by='집행금액', ascending=False).head(3)

In [ ]:
# 2018년
df_groups[3].sort_values(by='집행금액', ascending=False).head(3)

### 3. 회당 집행금액이 가장 많은 식당
(30회 이상 방문, 전체 공무원)

In [ ]:
for i, df_group in enumerate(df_groups):
    df_groups[i] = df_group[df_group['cnt']>30]

In [ ]:
# 전체
df_groups[0].sort_values(by='회당집행금액', ascending=False).head(3)

In [ ]:
# 2016년
df_groups[1].sort_values(by='회당집행금액', ascending=False).head(3)

In [ ]:
# 2017년 
df_groups[2].sort_values(by='회당집행금액', ascending=False).head(3)

In [ ]:
# 2018년
df_groups[3].sort_values(by='회당집행금액', ascending=False).head(3)

### 4. 인당 집행금액이 가장 많은 식당

In [ ]:
# 전체
df_groups[0].sort_values(by='인당집행금액', ascending=False).head(3)

In [ ]:
# 2016
# 김영란법 생기기 전인가? 3만원이 넘는다. 
df_groups[1].sort_values(by='인당집행금액', ascending=False).head(3)

In [ ]:
# 2017
df_groups[2].sort_values(by='인당집행금액', ascending=False).head(3)

In [ ]:
# 2018
df_groups[3].sort_values(by='인당집행금액', ascending=False).head(3)

### 5. 가장 많은 사람이 간 식당

In [ ]:
# 전체
df_groups[0].sort_values(by='대상인원', ascending=False).head(3)

In [ ]:
# 2016
df_groups[1].sort_values(by='대상인원', ascending=False).head(3)

In [ ]:
# 2017
df_groups[2].sort_values(by='대상인원', ascending=False).head(3)

In [ ]:
# 2018
df_groups[3].sort_values(by='대상인원', ascending=False).head(3)

### 6. 단체손님이 많이 가는 식당
8명 이상의 단체손님이 가는 식당 중 가장 자주 가는 식당

In [ ]:
# 전체
df_groups[0][df_groups[0]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

In [ ]:
# 2016년
# 2016년엔 단체로 가지 않은 것일까, 데이터가 적은 것일까
df_groups[1][df_groups[1]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

In [ ]:
# 2017년
df_groups[2][df_groups[2]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

In [ ]:
# 2018년
# 2016, 2017년에 사랑받던 제일가든이 없어지고
# 열빈이 회식 대세인가보다
df_groups[3][df_groups[3]['회당명수']>7].sort_values(by='cnt', ascending=False).head(3)

## 부서별로 살펴보기

In [ ]:
# 부서별로 묶기
def make_divs(df):
    df_divs = df.groupby('1소속부서')['cnt', 'num', '집행금액']\
            .sum().reset_index()
    # 인당/회당 금액 내기
    df_divs['인당집행금액'] = df_divs['집행금액'] / df_divs['num']
    df_divs['회당집행금액'] = df_divs['집행금액'] / df_divs['cnt']
    df_divs['인당집행금액'] = round(df_divs['인당집행금액']).astype(int)
    df_divs['회당집행금액'] = round(df_divs['회당집행금액']).astype(int)
    return df_divs

df_divs=[]
for i, df in enumerate(dfs):
    df_divs.append(make_divs(df))

### 1. 가장 회식을 자주 한 부서

In [ ]:
# 전체
df_divs[0].sort_values(by='cnt', ascending=False).head(3)

In [ ]:
# 2016
df_divs[1].sort_values(by='cnt', ascending=False).head(5)

In [ ]:
# 2017
df_divs[2].sort_values(by='cnt', ascending=False).head(5)

In [ ]:
# 2018
# 행정국 꾸준히 1위
df_divs[3].sort_values(by='cnt', ascending=False).head(5)

### 2. 가장 돈을 많이 쓴 부서

In [ ]:
# 전체
df_divs[0].sort_values(by='집행금액', ascending=False).head(3)

In [ ]:
# 2016
df_divs[1].sort_values(by='집행금액', ascending=False).head(3)

In [ ]:
# 2017
df_divs[2].sort_values(by='집행금액', ascending=False).head(3)

In [ ]:
# 2018
df_divs[3].sort_values(by='집행금액', ascending=False).head(3)

### 3. 가장 비싼 것을 먹는 부서

In [ ]:
# 전체
df_divs[0].sort_values(by='인당집행금액', ascending=False).head(3)
# 시장보다 부시장님들이 더 비싼 것을 먹는다!
# 시장님은 직원들 격려 피자를 자주 사주셔서 평균 금액이 내려간듯...
# 부시장은 1명이라 총금액은 적지만, 비싼 것을 먹는당

In [ ]:
# 2016년
# 역시 김영란법 시행 전이라 3만원이 넘는다. 
df_divs[1].sort_values(by='인당집행금액', ascending=False).head(3)

In [ ]:
# 2017년
# 라고 생각했는데 시행 후에도 3만원 넘는것 무엇...
df_divs[2].sort_values(by='인당집행금액', ascending=False).head(3)

In [ ]:
# 2018년
# 인당 7만 5천원?? 나중에 부시장 데이터 꼼꼼히 살펴보기
df_divs[3].sort_values(by='인당집행금액', ascending=False).head(3)

### 4. 가장 싼 것을 먹는 부서

In [ ]:
# 전체
df_divs[0].sort_values(by='인당집행금액', ascending=True).head(3)
# 인당 6000원대라니 8배 차이나는 이 빈부격차 무엇..

In [ ]:
# 2016년
df_divs[1].sort_values(by='인당집행금액', ascending=True).head(3)

In [ ]:
# 2017년
df_divs[2].sort_values(by='인당집행금액', ascending=True).head(3)

In [ ]:
# 2018년
# 재무국이 꾸준하게 하위권. 시의 예산을 보면 차마 비싼 것을 먹을 수 없는 것일까
df_divs[3].sort_values(by='인당집행금액', ascending=True).head(3)